## <b>Name :</b> Dharmik Vara  
## <b>Student ID :</b> 24215287

### <b>Dataset :</b> Dublin Rental Database

http://mlg.ucd.ie/modules/python/assignment1/rental/index.html

In [1]:
import requests
from requests.exceptions import RequestException
from bs4 import BeautifulSoup

### Collecting all the URLs present in the index page

In [2]:
index_url = 'http://mlg.ucd.ie/modules/python/assignment1/rental/index.html'
response = requests.get(index_url)
soup = BeautifulSoup(response.content, 'html.parser')
pages = soup.find_all('a', href = True)
for page in pages:
    print(page['href'])

index.html
Q1-page01.html
Q2-page01.html
Q3-page01.html
Q4-page01.html


In [3]:
pages[1]['href']

'Q1-page01.html'

### Creating the URLs for all quarter data

In [4]:
url_q1 = '/'.join(index_url.split('/')[:-1]) + '/' + pages[1]['href']
url_q2 = '/'.join(index_url.split('/')[:-1]) + '/' + pages[2]['href']
url_q3 = '/'.join(index_url.split('/')[:-1]) + '/' + pages[3]['href']
url_q4 = '/'.join(index_url.split('/')[:-1]) + '/' + pages[4]['href']

url_lists = [url_q1, url_q2, url_q3, url_q4]

for urls in url_lists:
    print(urls)

http://mlg.ucd.ie/modules/python/assignment1/rental/Q1-page01.html
http://mlg.ucd.ie/modules/python/assignment1/rental/Q2-page01.html
http://mlg.ucd.ie/modules/python/assignment1/rental/Q3-page01.html
http://mlg.ucd.ie/modules/python/assignment1/rental/Q4-page01.html


### Collecting the data from every page in each quarter and saving it in form of a dictionary

In [5]:
data = {
    'Quarter':[],
    'Month':[],
    'Price':[],
    'Property Type':[],
    'Location':[],
    'Bedrooms':[],
    'Bathrooms':[],
    'Parking':[],
    'Garden':[],
    'Lease Length':[],
    'Contact':[]
}

# To extract data for each quarter
for i in range(0,len(url_lists)):
    url = url_lists[i]
    print(url)
    page = 1
    while True: 
        # Modifiying URL to getch data from different pages
        if page < 10:
            url = url[:-6] + str(page) + '.html' # Till page number 10
        else:
            url = url[:-7] + str(page) + '.html' # From page number 10 
        try:
            response = requests.get(url)
            response.raise_for_status()
        except RequestException as e:
            print('All Pages Done!!')
            break
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # To append data into different dictionary keys
        for rec in soup.find_all('span', class_='record'):
            data['Quarter'].append('Q' + str(i+1))
            data['Month'].append(rec.get_text())
        
        # The data is in the tabular form and hence content from each row needs to be extracted and extra spaces need to be stripped
        for table in soup.find_all('table'):
            rows = table.find_all('tr')
            data['Price'].append( rows[0].get_text().split(':')[1].replace('\n','') )
            data['Property Type'].append( rows[1].get_text().split(':')[1].replace('\n','') )
            data['Location'].append( rows[2].get_text().split(':')[1].replace('\n','') )
            data['Bedrooms'].append( rows[3].get_text().split(':')[1].replace('\n','') )
            data['Bathrooms'].append( rows[4].get_text().split(':')[1].replace('\n','') )
            data['Parking'].append( rows[5].get_text().split(':')[1].replace('\n','') )
            data['Garden'].append( rows[6].get_text().split(':')[1].replace('\n','') )
            data['Lease Length'].append( rows[7].get_text().split(':')[1].replace('\n','') )
            data['Contact'].append( rows[8].get_text().split(':')[1].replace('\n','') )
        page += 1
        

http://mlg.ucd.ie/modules/python/assignment1/rental/Q1-page01.html


All Pages Done!!
http://mlg.ucd.ie/modules/python/assignment1/rental/Q2-page01.html
All Pages Done!!
http://mlg.ucd.ie/modules/python/assignment1/rental/Q3-page01.html
All Pages Done!!
http://mlg.ucd.ie/modules/python/assignment1/rental/Q4-page01.html
All Pages Done!!


### Checking number of data points in each key to verify if all the fields have same number of data

In [6]:
for k in list(data.keys()):
    print(f'{k} : {len(data[k])}')

Quarter : 1950
Month : 1950
Price : 1950
Property Type : 1950
Location : 1950
Bedrooms : 1950
Bathrooms : 1950
Parking : 1950
Garden : 1950
Lease Length : 1950
Contact : 1950


### Respresenting the dictionary data into a Data Frame

In [7]:
import pandas as pd 

data = pd.DataFrame(data)
data = data.apply(lambda x : x.str.strip())
data.head()

,Quarter,Month,Price,Property Type,Location,Bedrooms,Bathrooms,Parking,Garden,Lease Length,Contact
0,Q1,January 2024,"€ 7,200",Apartment,Dublin City South - Dublin 2,3,1 Bathroom,Yes,No,3 months,Estate Agent
1,Q1,January 2024,"€ 2,960 per month",Apartment,Dublin City South - Dublin 24,2 Bedrooms,2 Bathrooms,Yes,???,3 months,Estate Agent
2,Q1,January 2024,"€1,920.00 per month",Apartment,Dublin City South - Dublin 24,2 Bedrooms,2 Bathrooms,No,No,12 months,Estate Agent
3,Q1,January 2024,"€ 2,590",Apartment,Dublin City South - Dublin 6,2 Bedrooms,1 Bathroom,No,No,12 months,Estate Agent
4,Q1,January 2024,"€ 4,360 per month",House,Dublin City North - Dublin 1,3,2 Bathrooms,Yes,Yes,6 months,Estate Agent


### Saving the data in CSV format

In [8]:
data.to_csv('dublin_rental_data.csv', index = False)